In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

import json
import requests
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = URL(
    user="ABUHTAREVICH",
    password="",
    account="bw55606.us-central1.gcp",
    warehouse="COMPUTE_WH",
    database="",
    schema="",
    #role='xxxxx',
    #authenticator='https://xxxxx.okta.com',
)
engine = create_engine(url)
connection = engine.connect()

In [3]:
# show all tables

pd.set_option('display.max_rows', 1515)

In [6]:
query = '''
with 
users_installs as (
select adid
from events
where app_id = 'com.severex.cross.word.puzzle.logic'
and to_date(created_at) between '2023-03-02' and '2023-03-15'
and activity_kind = 'install'
and country in ('us')
and app_version_short = '2.2'
and network_name = 'Applovin'
and campaign_name = 'AL_CC_GP_ROAS_08.02.23 (8daa7da17f0728da0291fa43d7924640)'
group by 1
),

users_test as (
select adid
from events
where app_id = 'com.severex.cross.word.puzzle.logic'
    and adid in (select adid from users_installs)
    and datediff(day, to_date(installed_at), to_date(created_at)) between 0 and 7
    and event_name = 'ab_test'
    and parse_json(publisher_parameters):group_name::string in ('ab_sbs_on_classic_mode_off', 'ab_sbs_off_classic_mode_on', 'ab_sbs_on_classic_mode_on')
)

select
    adid,
    created_at,
    event_name,
    parse_json(publisher_parameters):lvl_total::integer lvl_total,
    parse_json(publisher_parameters):themeName::string theme,
    parse_json(publisher_parameters):lvl::integer lvl,
    parse_json(publisher_parameters):count_word_fail::integer count_word_fail,
    parse_json(publisher_parameters):type::string type,
    parse_json(publisher_parameters):answer::string answer,
    parse_json(publisher_parameters):result::string result,
    parse_json(publisher_parameters):time_round::integer time_round,
    parse_json(publisher_parameters):time_round::integer time_round2
from events
where app_id = 'com.severex.cross.word.puzzle.logic'
and datediff(day, to_date(installed_at), to_date(created_at)) between 0 and 45
and adid in (select adid from users_installs)
and adid not in (select adid from users_test)
and event_name in ('game_start', 'game_win', 'game_word_end', 'game_hint_spend', 
                                                    'game_word_fail')
and parse_json(publisher_parameters):lvl_total::integer < 51
'''

In [30]:
query = '''
with 
users_installs as (
select adid
from events
where app_id = '1636821805'
and to_date(created_at) between '2023-03-02' and '2023-03-15'
and activity_kind = 'install'
and country in ('us')
and app_version_short = '2.2'
and network_name = 'AppLovin_Tracking'
and campaign_name = 'AL_FG_IOS_US_ROAS_07.11.22 (19409ae1e5c147d6d3fcd12389d62e49)'
group by 1
),

users_test as (
select adid
from events
where app_id = '1636821805'
    and adid in (select adid from users_installs)
    and datediff(day, to_date(installed_at), to_date(created_at)) between 0 and 7
    and event_name = 'ab_test'
    and parse_json(publisher_parameters):group_name::string in ('ab_sbs_on_classic_mode_off', 'ab_sbs_off_classic_mode_on', 'ab_sbs_on_classic_mode_on')
)

select
    adid,
    created_at,
    event_name,
    parse_json(publisher_parameters):lvl_total::integer lvl_total,
    parse_json(publisher_parameters):themeName::string theme,
    parse_json(publisher_parameters):lvl::integer lvl,
    parse_json(publisher_parameters):count_word_fail::integer count_word_fail,
    parse_json(publisher_parameters):type::string type,
    parse_json(publisher_parameters):answer::string answer,
    parse_json(publisher_parameters):result::string result,
    parse_json(publisher_parameters):time_round::integer time_round,
    parse_json(publisher_parameters):time_round::integer time_round2
from events
where app_id = '1636821805'
and datediff(day, to_date(installed_at), to_date(created_at)) between 0 and 45
and adid in (select adid from users_installs)
and adid not in (select adid from users_test)
and event_name in ('game_start', 'game_win', 'game_word_end', 'game_hint_spend', 
                                                    'game_word_fail')
and parse_json(publisher_parameters):lvl_total::integer < 51
'''

In [31]:
df = pd.read_sql(query, connection)

In [32]:
df.head()

,adid,created_at,event_name,lvl_total,theme,lvl,count_word_fail,type,answer,result,time_round,time_round2
0,f41fd3d8fd7a6c5ae517d82e8abc2d23,1678765580,game_word_end,42,Botany,3,NaN,words,FIRE,None,NaN,NaN
1,f41fd3d8fd7a6c5ae517d82e8abc2d23,1678765582,game_word_end,42,Botany,3,NaN,words,ROWAN,None,NaN,NaN
2,b10624afb4331b5a6b2278a161ced245,1678765583,game_word_fail,25,Arts,1,NaN,words,LISTED,DINNER,NaN,NaN
3,f41fd3d8fd7a6c5ae517d82e8abc2d23,1678765583,game_word_end,42,Botany,3,NaN,words,ENTIRE,None,NaN,NaN
4,f41fd3d8fd7a6c5ae517d82e8abc2d23,1678765584,game_win,42,Botany,3,0.0,None,None,None,34.0,34.0


In [33]:
df.dtypes

adid                object
created_at          object
event_name          object
lvl_total            int64
theme               object
lvl                  int64
count_word_fail    float64
type                object
answer              object
result              object
time_round         float64
time_round2        float64
dtype: object

In [34]:
df.event_name.value_counts()

game_word_fail     294583
game_word_end      292459
game_start          53280
game_win            52961
game_hint_spend     24085
Name: event_name, dtype: int64

In [35]:
df.theme = df.theme.replace({'Archaeology': 'Archeology'})

In [36]:
df_starts = df.query('event_name == "game_start"').groupby(['lvl_total', 'theme', 'lvl'], as_index=False) \
                                      .agg({'adid': 'nunique'}) \
                                      .rename(columns={'adid': 'users_starts'}) \
                                      .sort_values('users_starts', ascending=False)

In [37]:
df_ends = df.query('event_name == "game_win"').groupby(['lvl_total', 'theme', 'lvl'], as_index=False) \
                                      .agg({'adid': 'nunique', 'count_word_fail': 'mean',
                                            'time_round': 'mean', 'time_round2': 'median'}) \
                                      .rename(columns={'adid': 'users_ends', 'time_round': 'time_mean',
                                                       'time_round2': 'time_median'}) \
                                      .sort_values('users_ends', ascending=False)

df_ends['time_mean'] = round(df_ends.time_mean)

In [38]:
df_total = df_ends.merge(df_starts)

In [39]:
df_final = df_total[['lvl_total', 'theme', 'lvl', 'users_starts', 'users_ends', 'count_word_fail',
                     'time_mean', 'time_median']].copy()

In [40]:
df_hint_count = df.query('event_name == "game_hint_spend"').groupby(['lvl_total', 'theme', 'lvl'], as_index=False) \
                                      .agg({'adid': 'nunique', 'event_name': 'count'}) \
                                      .rename(columns={'adid': 'users_hint'}) \
                                      .sort_values('users_hint', ascending=False)

In [41]:
df_final_last = df_final.merge(df_hint_count, on=['lvl_total', 'theme', 'lvl'])
df_final_last['ending'] = df_final_last.users_ends / df_final_last.users_starts
df_final_last['churn_start'] = df_final_last.users_starts / df_final_last.users_starts.max()
df_final_last['churn_end'] = df_final_last.users_ends / df_final_last.users_starts.max()
df_final_last['churn_end_lagged'] = df_final_last['churn_end'].shift(1)
df_final_last['churn_between_lvls'] = df_final_last.churn_end_lagged - df_final_last.churn_start
df_final_last['churn_in_lvl'] = df_final_last.churn_start - df_final_last.churn_end
df_final_last['users_used_hint_per_started'] = df_final_last.users_hint / df_final_last.users_starts
df_final_last['game_hint_spend_per_user_used'] = df_final_last.event_name / df_final_last.users_hint

In [42]:
df_final_copy = df_final_last[['lvl_total', 'users_starts', 'ending', 'churn_start',
               'churn_end', 'churn_between_lvls', 'churn_in_lvl', 'count_word_fail']].copy()

In [43]:
df_final_copy.query('lvl_total < 51 & users_starts > 100')

,lvl_total,users_starts,ending,churn_start,churn_end,churn_between_lvls,churn_in_lvl,count_word_fail
0,1,4115,0.979587,1.000000,0.979587,NaN,0.020413,1.304016
1,2,3772,0.899523,0.916646,0.824544,0.062940,0.092102,3.382114
2,3,3244,0.875462,0.788335,0.690158,0.036209,0.098177,10.280103
3,4,2707,0.972294,0.657837,0.639611,0.032321,0.018226,2.190792
4,5,2573,0.945200,0.625273,0.591009,0.014338,0.034265,4.819373
5,6,2346,0.973998,0.570109,0.555286,0.020899,0.014824,2.195752
6,7,2219,0.937359,0.539247,0.505468,0.016039,0.033779,8.139189
7,8,2011,0.886624,0.488700,0.433293,0.016768,0.055407,4.528084
8,9,1734,0.917532,0.421385,0.386634,0.011908,0.034751,12.422571
9,10,1550,0.974839,0.376671,0.367193,0.009964,0.009478,5.437657


In [44]:
df.query('event_name == "game_win" & lvl_total == 35').adid.nunique()

464

In [45]:
8122-7634

488

In [46]:
df_time = df.query('event_name == "game_win" & time_round != 0') \
                                      .groupby(['lvl_total'], as_index=False) \
                                      .agg({'time_round': 'median'})

df_time.head()

,lvl_total,time_round
0,1,43.0
1,2,57.0
2,3,107.0
3,4,38.0
4,5,75.5


In [47]:
df_time

,lvl_total,time_round
0,1,43.0
1,2,57.0
2,3,107.0
3,4,38.0
4,5,75.5
5,6,63.0
6,7,100.5
7,8,90.0
8,9,117.5
9,10,86.0


### Слова вопросы ответы с ошибками

In [23]:
df_fail = df.query('event_name == "game_word_fail" & type == "words"') \
    .groupby(['lvl_total', 'theme', 'lvl', 'answer'], as_index=False) \
                                      .agg({'adid': 'nunique', 'created_at': 'count'}) \
                                      .rename(columns={'adid': 'users_fail', 'created_at': 'events_cnt'})

df_fail['fails_per_user'] = df_fail.events_cnt / df_fail.users_fail
df_fail.head()

,lvl_total,theme,lvl,answer,users_fail,events_cnt,fails_per_user
0,1,Elementary,1,ONLY,2,2,1.0
1,1,Elementary,1,SIGN,2,5,2.5
2,1,Elementary,1,YELLOW,1,2,2.0
3,1,Interesting,1,EMAIL,1,2,2.0
4,1,Interesting,1,ENERGY,2,4,2.0


In [24]:
df_final_starts = df_final_last.query('users_starts > 100 & lvl_total < 201').iloc[:,[0,1,2,3]].copy()

In [25]:
df_final_fail = df_final_starts.merge(df_fail, how='left', on=['lvl_total', 'theme', 'lvl']).copy()

df_final_fail['users_fail_perc'] = df_final_fail.users_fail / df_final_fail.users_starts
df_final_fail['events_cnt'] = df_final_fail.events_cnt / df_final_fail.users_starts

In [26]:
df_final_fail.shape

(1707, 9)

In [27]:
df_last_final_fail = df_final_fail[['lvl_total', 'theme', 'lvl', 'answer', 'users_fail_perc', 'fails_per_user', 'events_cnt']].copy()

In [31]:
df_last_final_fail

,lvl_total,theme,lvl,answer,users_fail_perc,fails_per_user,events_cnt
0,1,Interesting,1,EMAIL,0.000016,2.000000,0.000032
1,1,Interesting,1,ENERGY,0.000032,2.000000,0.000064
2,1,Interesting,1,FORUM,0.000016,1.000000,0.000016
3,1,Interesting,1,LAWYER,0.000016,5.000000,0.000080
4,1,Interesting,1,LINK,0.000016,2.000000,0.000032
...,...,...,...,...,...,...,...
1702,200,Food,1,STALKER,0.175325,2.481481,0.435065
1703,200,Food,1,STARE,0.113276,2.133758,0.241703
1704,200,Food,1,SUNY,0.193362,3.492537,0.675325
1705,200,Food,1,SWAY,0.279942,4.541237,1.271284


In [34]:
df_old_final_fail = pd.read_csv('answers_200.csv') 

In [37]:
df_old_final_fail_1 = df_old_final_fail.iloc[:,[0,1,2,4]].copy()

In [39]:
df_old_final_fail_1.columns = ('lvl_total', 'theme', 'lvl', 'answer')

In [42]:
df_old_final_fail_1.merge(df_last_final_fail, how='left', on=['lvl_total', 'theme', 'lvl', 'answer']).fillna(0)

,lvl_total,theme,lvl,answer,users_fail_perc,fails_per_user,events_cnt
0,1,Interesting,1,ONLY,0.175874,3.323668,0.584548
1,1,Interesting,1,SIGN,0.144511,2.628540,0.379852
2,1,Interesting,1,WIN,0.047158,2.441017,0.115113
3,1,Interesting,1,YEAR,0.051985,1.508610,0.078426
4,1,Interesting,1,YES,0.046934,2.608311,0.122418
5,2,Interesting,2,ACCOUNT,0.158115,2.178254,0.344415
6,2,Interesting,2,CASE,0.213932,2.835943,0.606699
7,2,Interesting,2,CREATION,0.080991,2.018038,0.163442
8,2,Interesting,2,FORCE,0.201033,3.105193,0.624247
9,2,Interesting,2,INTERNET,0.126795,3.464522,0.439284


In [ ]:
query = '''
with 
users_installs as (
select 
    adid
from events
where app_id = '1636821805'
    and to_date(created_at) between '2023-01-06' and '2023-01-23'
    and activity_kind = 'install'
    and country = 'us'
    and app_version_short in ('1.8', '1.9')
)

select
    adid,
    event_name,
    parse_json(publisher_parameters):lvl_total::integer lvl_total
from events
where app_id = '1636821805'
and datediff(day, to_date(installed_at), to_date(created_at)) between 7 and 7
and adid in (select adid from users_installs)
and event_name in ('game_start')
'''

In [ ]:
df = pd.read_sql(query, connection)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.adid.nunique()

In [ ]:
df_lvl_max = df.groupby('adid', as_index=False) \
    .agg({'lvl_total': max})

df_lvl_max.head()

In [ ]:
df_lvl_max.lvl_total.describe()

### JSON SERVER

In [96]:
# hosted your json response as a url response 
link = 'https://wordpuzzle.europosit.com/db/themes.json'

# getting requests from the server
req = requests.get(link)
text_data = req.text
json_dict = json.loads(text_data)

# converting the json dictionary to a dataframe
df_json = pd.DataFrame.from_dict(json_dict["themes"], orient='index')

In [98]:
df_json.head()

,key,prior,name,icon,levels,lastupdate,medium_icon,large_icon,publish
0,0,0,Interesting,themes.themes.0.icon.file,"{'0': {'key': 0, 'prior': 0, 'phrase': 'Coca-C...",1673542104001,themes.themes.0.medium_icon.file,themes.themes.0.large_icon.file,True
1,1,1,Around the World,themes.themes.1.icon.file,"{'0': {'key': 0, 'prior': 0, 'phrase': 'You ca...",1669268820736,themes.themes.1.medium_icon.file,themes.themes.1.large_icon.file,True
2,2,2,Health,themes.themes.2.icon.file,"{'0': {'key': 0, 'prior': 0, 'phrase': 'When w...",1667313729083,themes.themes.2.medium_icon.file,themes.themes.2.large_icon.file,True
3,3,3,History,themes.themes.3.icon.file,"{'0': {'key': 0, 'prior': 0, 'phrase': 'Damasc...",1667313745044,themes.themes.3.medium_icon.file,themes.themes.3.large_icon.file,True
4,4,4,Geography,themes.themes.4.icon.file,"{'0': {'key': 0, 'prior': 0, 'phrase': 'Icelan...",1669796578135,themes.themes.4.medium_icon.file,themes.themes.4.large_icon.file,True


In [99]:
df_json.dtypes

key             int64
prior           int64
name           object
icon           object
levels         object
lastupdate      int64
medium_icon    object
large_icon     object
publish          bool
dtype: object

In [121]:
with open('themes.rtf', 'r') as file:
    text = file.read()
print(text)

{\rtf1\ansi\ansicpg1252\cocoartf2706
\cocoatextscaling0\cocoaplatform0{\fonttbl\f0\fswiss\fcharset0 Helvetica;}
{\colortbl;\red255\green255\blue255;}
{\*\expandedcolortbl;;}
\paperw11900\paperh16840\margl1440\margr1440\vieww11520\viewh8400\viewkind0
\pard\tx566\tx1133\tx1700\tx2267\tx2834\tx3401\tx3968\tx4535\tx5102\tx5669\tx6236\tx6803\pardirnatural\partightenfactor0

\f0\fs24 \cf0 \{\
  "themes": \{\
    "0": \{\
      "key": 0,\
      "prior": 0,\
      "name": "Interesting",\
      "icon": "themes.themes.0.icon.file",\
      "levels": \{\
        "0": \{\
          "key": 0,\
          "prior": 0,\
          "phrase": "Coca-Cola was originally green. And it's false! Coca-Cola has always been the same color since its invention in 1886. Like many people who change history, John Pemberton, an Atlanta pharmacist, was inspired by simple curiosity. One afternoon, he stirred up a fragrant, caramel-colored liquid and, when it was done, he carried it a few doors down to Jacobs\'92 Pharmacy.